# Algemeen

De implementatie van een programmeertaal in TESTed bestaat uit twee grote onderdelen:

- De configuratieklasse
- De sjablonen

Aan de hand van de opties en callbacks in de configuratieklasse zal TESTed met de toepasselijke sjablonen de uit te voeren code genereren, waarna het compilatie- en uitvoeringscommando aan de configuratieklasse gevraagd wordt.

Het is bij het implementeren van een programmeertaal nuttig om dus volgende korte stappenplan in het achterhoofd te houden:

1. TESTed roept enkele functies aan uit de configuratieklasse om opties voor de programmeertaal op te vragen.
2. TESTed gebruikt de sjablonen om de juiste code te genereren.
3. TESTed roept de compilatie- en uitvoeringsfunctie op van de configuratieklasse, die het commando om te compileren en uit te voeren teruggeeft.

Het is ook nuttig om een voorbeeldoefening op te stellen en gebruik te maken van de module `tested.manual`. Hierbij kan een oefening uitgevoerd worden, waarbij TESTed veel logs uitschrijft. Dit is handig om te weten waar dingen verkeerd lopen.

# De programmeertaal C

In deze paragraaf voegen we de programmeertaal C toe aan TESTed.

## Ondersteunde functionaliteit

Voor we van start gaan, overlopen we eerst kort wat we willen ondersteunen.
Uiteraard is dit zoveel mogelijk, maar vooral op het vlak van gegevenstypes moeten we soms keuzes maken.

Welke basistypes gaan we niet ondersteunen?

- `sequence` - Arrays zijn een speciaal geval in C: statische arrays kunnen bijvoorbeeld niet als returnwaarde dienen, en ook als functieargument zijn ze niet ideaal. Dynamische arrays nemen de vorm aan van een pointer en een grootte. Doordat dit twee waarden zijn, ondersteunt TESTed dat momenteel niet.
- `set` - C heeft geen ingebouwde verzamelingen.
- `map` - C heeft geen ingebouwde map of dict. Er zijn wel structs, maar daarvan is het niet mogelijk om at runtime de velden te achterhalen, wat ze niet bruikbaar maakt.

Welke geavanceerde types gaan we niet ondersteunen?

- `big_int` - C heeft geen ingebouwd type voor getallen van arbitraire grootte.
- `fixex_precision`- C heeft geen ingebouwd type voor kommagetallen me arbitraire precisie.
- Andere datastructuren, zoals `array` en `list` (voor de redenen van hierboven). Ook `tuple` wordt niet ondersteund, omdat het niet bestaat in C.

## Configuratieklasse

De eerste stap in het toevoegen van een programmeertaal is het implementeren van de configuratieklasse.
Maak hiervoor een nieuw Python-bestand in `test/langauges/c/config.py`.
Hierin moet een klasse komen die `Language` uitbreidt:

In [ ]:
from tested.languages import Language

class CConfig(Language):
    pass

De klasse `Language` heeft veel functies, waarvan een deel verplicht te implementeren is.
Hieronder overlopen we de functies per onderwerp.

### Programmeertaaleigenschappen

Als eerste zijn er enkele functies die weinig uitleg behoeven, en enkele eigenschappen van C implementeren.


In [ ]:
def file_extension(self) -> str:
        return "c"

De bestandsextensie voor bestanden in C. Er zijn in C ook headers (`.h`), maar daar moeten we geen rekening mee houden.


In [ ]:
def submission_name(self, plan: Plan) -> str:
    return "submission"

De naam van het bestand met de oplossing van de student.

In [ ]:
def selector_name(self) -> str:
    return "selector"

def needs_selector(self):
    return True

De naam van de selector voor batchcompilatie en of dat een selector nodig is (in programmeertalen zoals Python is dat niet nodig).

In [ ]:
def context_name(self, tab_number: int, context_number: int) -> str:
    return f"context_{tab_number}_{context_number}"

De naam van een bestand voor de context van een specifiek tabblad.

In [ ]:
def conventionalise_function(self, function_name: str) -> str:
    return decamelize(function_name)

Deze functie zet namen uit het testplan om naar de namen volgens de gebruiken van de taal. In C kiezen we ervoor om functies in `snake_case` te zetten.

### Compileren en uitvoeren van code

De eerste twee functies die we gaan bekijken zorgen ervoor dat de code die TESTed gegenereerd heeft, gecompileerd en uitgevoerd kan worden.

Als eerste bekijken we het compileren:

In [ ]:
def generation_callback(self, files: List[str]) -> CallbackResult:
    main_file = files[-1]
    exec_file = Path(main_file).stem
    result = executable_name(exec_file)
    return ["gcc", "-std=c11", "-Wall", "values.c", main_file, "-o", result], [result]

Zoals de naam doet vermoeden wordt deze functie opgeroepen nadat TESTed de code heeft gegenereerd.
Als argument wordt een lijst van bestanden meegegeven die gecompileerd moeten worden tot één uitvoerbaar geheel.
Per conventie is het laatste bestand in de lijst het bestand met de `main`-functie.
Normaliter worden al deze bestanden in het compilatiecommando opgenomen, maar in het geval van C willen we dat niet: dit voorkomt dubbele definities.

Het eerste dat we doen is het bepalen van de naam van de executable.
De functie `executable_name` is een hulpfunctie die de juiste extensie aan een string plakt, afhankelijk van het platform (niets op Linux, `exe` op Windows).

Als returnwaarde verwacht deze functie een tuple met twee elementen:

- Het uit te voeren commando voor de compilatie. In dit geval gebruiken we GCC. Als opties geven we `std=c11` en `Wall` mee. Dit is uiteraard te kiezen per programmeertaal.
- Een lijst van resulterende bestanden. Bij C is er maar één bestand: de executable.

Een concreet voorbeeld maakt dit duidelijker. Stel dat we een oefening met één context compileren in batchcompilatie:

In [ ]:
>>> generation_callback(["submission.c", "context0.c", "selector.c"])
(["gcc", "-std=c11", "-Wall", "selector.c", "-o", "selector.exe"], ["selector.exe"])

Nu de code gecompileerd is, moeten we hem nog steeds kunnen uitvoeren. Dit gebeurt met een gelijkaardige functie:


In [ ]:
def execution_command(self,
                      cwd: Path,
                      file: str,
                      arguments: List[str]) -> List[str]:
    local_file = cwd / executable_name(Path(file).stem)
    return [str(local_file.absolute()), *arguments]

Deze functie heeft meer argumenten:

- `cwd` of _current working directory_, de map waar de uitvoering zal plaatsvinden.
- `file`, het bestand dat uitgevoerd moet worden (het bestand met de `main`-functie). In het geval van C zal dat altijd de executable zijn.
- `arguments`, argumenten die aan het programma meegegeven moeten worden.

Aangezien C compileert tot een uitvoerbaar bestand, kunnen we dat bestand hier direct oproepen. Het enige dat we doen is opnieuw de juiste extensie toevoegen en een absoluut pad gebruiken. Dat laatst omdat het commando door TESTed met `subprocess` uitgevoerd wordt, en dus het pad naar het uitvoerbaar bestand nodig heeft.

De returnwaarde is hier het uit te voeren commando.

### Aanpassen van de oplossingscode

In TESTed kunnen er meerdere `main`-functies zijn:

- De oplossing van de student kan een `main`-functie hebben.
- Zowel de contexten als de selector kunnen `main`-functies hebben.

C staat slechts één `main`-functie toe.

Een ander probleem is dat de selector elke context include (zoals we later zullen zien bij de sjablonen), en elke context ook de oplossing include.

Om deze redenen moeten we de code van de oplossing een beetje aanpassen:

- We voegen aan `guard` toe, zodat de oplossing eenmalig geladen wordt.
- We hernoemen de `main`-functie naar `solution_main` indien die bestaat.

In [ ]:
def solution_callback(self, solution: Path, plan: Plan):
    with open(solution, "r") as file:
        contents = file.read()
    with open(solution, "w") as file:
        header = "#ifdef INCLUDED\n#pragma once\n#endif\n\n"
        result = header + contents.replace("main", "solution_main")
        file.write(result)

### Ondersteunde functionaliteit

Bij de voorbereiding hebben we besproken welke functionaliteit we willen ondersteunen. Hier implementeren we dat:

In [ ]:

def type_support_map(self) -> Mapping[AllTypes, TypeSupport]:
    return fallback(super().type_support_map(), {
        ant.INT_8:            TypeSupport.SUPPORTED,
        ant.U_INT_8:          TypeSupport.SUPPORTED,
        ant.INT_16:           TypeSupport.SUPPORTED,
        ant.U_INT_16:         TypeSupport.SUPPORTED,
        ant.INT_32:           TypeSupport.SUPPORTED,
        ant.U_INT_32:         TypeSupport.SUPPORTED,
        ant.INT_64:           TypeSupport.SUPPORTED,
        ant.U_INT_64:         TypeSupport.SUPPORTED,
        ant.SINGLE_PRECISION: TypeSupport.SUPPORTED,
        ant.DOUBLE_EXTENDED:  TypeSupport.SUPPORTED,
        ant.FIXED_PRECISION:  TypeSupport.UNSUPPORTED,
        ast.ARRAY:            TypeSupport.SUPPORTED,
        ast.LIST:             TypeSupport.UNSUPPORTED,
        ast.TUPLE:            TypeSupport.UNSUPPORTED,
        bst.SET:              TypeSupport.UNSUPPORTED,
        bot.MAP:              TypeSupport.UNSUPPORTED,
    })

def supported_constructs(self) -> Constructs:
    return (Constructs.MAIN | Constructs.NOTHING | Constructs.FUNCTION_CALL
            | Constructs.ASSIGNMENT)

Eerst sommen we de ondersteunde en niet-ondersteunde types op. In de tweede functie geven we aan welke _constructs_ we ondersteunen.

## Sjablonen

De tweede stap is het implementeren van de sjablonen. Naast de verplichte sjablonen implementeren we ook een paar bijkomende:

- `context.c` - het sjabloon voor de contextcode te genereren
- `declaration.mako` - vertaal het type van een variabele naar code
- `function.mako`-  vertaalt een functieoproep
- `selector.c` - het sjabloon voor de selecter voor batchcompilatie
- `statement.c` - vertaalt een statement of expressie naar code
- `value.mako` - vertaalt een waarde naar code
- `value_arguments.mako` - hulpsjabloon voor `value`
- `value_basic.mako`- hulpsjabloon voor `value`

### Het contextsjabloon `context.c`

Dit is veruit het grootste en ingewikkeldste sjabloon. Het is verantwoordelijk om de testcode te genereren voor één context.

In [ ]:
<%! from tested.languages.generator import _TestcaseArguments %>
<%! from tested.serialisation import Statement, Expression %>
<%! from tested.utils import get_args %>

#include <stdio.h>

We importeren de values-module (hierover later meer) en de oplossing van de student.
De variabele `submission_name` zal de naam van het oplossingsbestand bevatten.

In [ ]:
#include "values.h"
#include "${submission_name}.c"

We importeren de programmeertaalspecifieke evaluatoren die nodig zijn.
TODO

In [ ]:
## Import the language specific evaluators we will need.
% for name in evaluator_names:
    #include "${name}.c"
% endfor

De bestanden waarin de return-channel opgeslagen wordt. We maken ook een bestand aan voor exceptions: C ondersteunt geen exceptions, maar TESTed verwacht toch een bestand.

In [ ]:
static FILE* value_file = NULL;
static FILE* exception_file = NULL;

Hulpfunctie, opgeroepen tussen de testgevallen om de uitvoer te scheiden.

In [ ]:
static void write_delimiter() {
    fprintf(value_file, "--${secret_id}-- SEP");
    fprintf(exception_file, "--${secret_id}-- SEP");
    fprintf(stdout, "--${secret_id}-- SEP");
    fprintf(stderr, "--${secret_id}-- SEP");
}

Dit zijn de standaardfuncties die gebruikt worden om een resultaat te verwerken. Normaal zijn dit deze functies:

- `send` - schrijf een waarde naar een bestand.
- `send_exception` - schrijf een exception naar een bestand.
- `send_specific_value` - schrijf het resultaat van een programmeertaalspecifieke evaluatie naar de return-channel.
- `send_specific_exception` - schrijf het resultaat van een programmeertaalspecifieke evaluatie naar de exception-channel.

In C wijken we daar lichtjes van af:

- De `send` is een macro, geen functie, omdat C geen `any` type heeft.
- De `exception`-functies implementeren we niet.

De implementatie van deze functies is eenvoudig: meestal wordt gewoon de overeenkomende functie uit de `values` module opgeroepen met het juiste bestand als parameter.

In [ ]:
#define send(value) write_value(value_file, value)

#pragma GCC diagnostic ignored "-Wunused-function"
static void send_specific_value(EvaluationResult r) {
    send_evaluated(value_file, r.result, r.readableExpected, r.readableActual, r.nrOfMessages, r.messages);
}
#pragma GCC diagnostic pop

Elke testgeval heeft twee functies evaluatiefuncties nodig:

- `v_evaluate_x` - verwerkt de waarde van een testgeval
- `e_evaluate_x` - verwerkt de exception van een testgeval

De inhoud van deze functies wordt door TESTed gegeven als variabele in `testcase.value_function`. Deze functie bevat ofwel de code om een programmeertaalspecifieke evaluator op te roepen, of bevat de code om een van de standaardfuncties die we hierboven gedefinieerd hebben op te roepen. Merk op dat we deze variabele meegeven aan het functiesjabloon, want deze variabele bevat een functieoproep, geëncodeerd in het serialisatieformaat.

Bij C implementeren we de exception-functie opnieuw niet.

In [ ]:
% for testcase in testcases:
    % if testcase.value_function:
        #define v_evaluate_${loop.index}(value) <%include file="statement.mako" args="statement=testcase.value_function"/>
    % endif
% endfor

Het eigenlijke uitvoeren van de testgevallen gebeurt in de functie hieronder.

In [ ]:

int ${context_name}() {

Het eerste dat we doen is de bestanden aanmaken.

In [ ]:
    value_file = fopen("${value_file}", "w");
    exception_file = fopen("${exception_file}", "w");

    ${before}

Eerst roepen we de `main`-functie van de oplossing op indien het testplan dat vereist.
TODO: argumenten

In [ ]:
    // Main functions are not support at the moment.
    // TODO: arguments?
    % if context_testcase.exists:
        solution_main();
    % endif

We schrijven de scheidingstekens uit naar de bestanden. We doen dat altijd, ook al kan het zijn dat het vorige testgeval niets uitgevoerd heeft.

In [ ]:
    write_delimiter();

Hieronder doen we de eigenlijke testgevallen. Elk testgeval heeft een statement als invoer. Indien dit statement een expression is en we zijn geïnteresseerd in de returnwaarde, moeten we natuurlijk de waarde opvangen. Dat gebeurt met `v_evaluate_${loop.index}(`. De `\` op het einde duidt aan dat het regeleinde weggelaten mag worden.

Na de elke testcase schrijven we ook opnieuw het scheidingsteken uit.

In [ ]:
    ## Generate the actual tests based on the context.
    % for testcase in testcases:
        ## If we have a value function, we have an expression.
        % if testcase.value_function:
            v_evaluate_${loop.index}(\
        % endif
        <%include file="statement.mako" args="statement=testcase.command" />\
        % if testcase.value_function:
            );
        % endif

        write_delimiter();

    % endfor

We sluiten ten slotte de bestanden.

In [ ]:
    ${after}

    fclose(value_file);
    fclose(exception_file);
    return 0;
}

Omdat TESTed zowel contextcompilatie als batchcompilatie ondersteunt, moet elke context een `main`-functie hebben. C laat slechts 1 `main`-functie toe. Indien we in batchcompilatie zitten, zal de selector gebruikt worden, en zal `INCLUDED` op `TRUE` staan. In dat geval voegen we geen `main`-functie toe.

In [ ]:
#ifndef INCLUDED
int main() {
    return ${context_name}();
}
#endif

## Het selectorsjabloon `selector.c`

Dit sjabloon is verantwoordelijk voor het uitvoeren van de juiste context aan de hand van een parameter.

De implementatie ervan is redelijk eenvoudig: we includen alle contexten, en definiëren een `main`-functie die het eerste argument vergelijkt met elke naam van de context en dan de juiste uitvoert.

Zoals besproken bij het contextsjabloon zetten we ook `INCLUDED` op `true`.

In [ ]:
#include <string.h>

#define INCLUDED true

% for c in contexts:
    #include "${c}.c"
% endfor

int main(int argc, const char* argv[]) {

    const char* name = argv[1];

    if (name == NULL) {
        fprintf(stderr, "You must select the context");
        return -2;
    }

    % for c in contexts:
        if (strcmp("${c}", name) == 0) {
            return ${c}();
        }
    % endfor
    return -1;
}


